# Titanic Dataset Analysis

In this notebook, we will perform an analysis of the Titanic dataset. The goal is to build predictive models to determine the survival of passengers.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# Load Dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

## Data Overview

In this section, we perform an initial exploration of the training dataset to understand its structure and contents.

In [ ]:
# Data Overview
print(train_df.head())
print(train_df.info())
print(train_df.describe())

In [ ]:
# Fill missing values
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

## Encode Categorical Variables

We use `LabelEncoder` to convert the categorical variables 'Sex' and 'Embarked' into numerical format. This step is necessary for machine learning algorithms to process the data. 

- **LabelEncoder**: Assigns a unique integer to each category (e.g., 'male' as 1, 'female' as 0).

The encoder is applied to both the training and test datasets for consistency.

In [1]:
# Encode categorical variables
label_encoder = LabelEncoder()
train_df['Sex'] = label_encoder.fit_transform(train_df['Sex'])
train_df['Embarked'] = label_encoder.fit_transform(train_df['Embarked'])
test_df['Sex'] = label_encoder.transform(test_df['Sex'])
test_df['Embarked'] = label_encoder.transform(test_df['Embarked'])

NameError: name 'LabelEncoder' is not defined

In [ ]:
# Normalize numerical features
scaler = StandardScaler()
train_df[['Age', 'Fare']] = scaler.fit_transform(train_df[['Age', 'Fare']])
test_df[['Age', 'Fare']] = scaler.transform(test_df[['Age', 'Fare']])

In [ ]:
# Feature Engineering
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

In [ ]:
# Data Visualization
sns.countplot(x='Survived', data=train_df)
plt.show()

sns.countplot(x='Pclass', hue='Survived', data=train_df)
plt.show()

sns.countplot(x='Sex', hue='Survived', data=train_df)
plt.show()

sns.histplot(train_df['Age'], bins=30, kde=True)
plt.show()

In [ ]:
# Model Building
X = train_df.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
y = train_df['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Model Evaluation
y_pred = model.predict(X_val)
print(f'Accuracy: {accuracy_score(y_val, y_pred)}')
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))